In [1]:
from config import *


In [2]:
print(f'{year}年{month}月')

2020年4月


In [3]:
import pandas as pd
import sys
sys.path.append('../../py')
import db
import weighted
import inspect
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
%matplotlib inline

In [4]:
data_original=pd.read_sql(sql=f"select * from _{year}{month:02} where monthly_salary>0 and monthly_salary<80000", con=db.get_conn())

In [5]:
data_original.shape

(119172, 114)

In [6]:
data = data_original

In [7]:
del data['publish_date']
del data['published_on_weekend']
del data['title']
del data['company_title']
del data['company_description']
del data['job_description']
del data['job_id']

KeyError: 'company_title'

In [ ]:
def get_sub_stats_by_prefix(data, prefix):
    
    features = [feature for feature in data.columns if feature.startswith(prefix)]
    salary_mean=[]
    salary_median=[]
    salary_95_min=[]
    salary_95_max=[]
    count=[]
    
    features_out=[]
    for feature in features:
        #print(feature)
        idata=data[data[feature]==1]
        headcount=idata.headcount.sum()
        values = idata.monthly_salary.values
        weights = idata.headcount.values
        #print(str(headcount))
        if headcount==0:
            continue
        
        salary_mean.append(weighted.weighted_mean(values, weights))
        q = weighted.weighted_quantile(values,[0.025,0.5,0.975],weights)
        salary_median.append(q[1])
        salary_95_min.append(q[0])
        salary_95_max.append(q[2])
        count.append(idata.headcount.sum())
        features_out.append(feature)
    sub_data=pd.DataFrame()
    sub_data['rank']=range(0,len(features_out))
    sub_data[prefix]=[f.replace(prefix,'') for f in features_out]
    sub_data['salary_mean']=salary_mean
    sub_data['salary_median']=salary_median
    sub_data['salary_95_min']=salary_95_min
    sub_data['salary_95_max']=salary_95_max
    sub_data['head_count']=count
    sub_data['percentage']=count/np.sum(count)
    sub_data=sub_data.sort_values(by='salary_mean', ascending=False)
    sub_data['rank']=range(1,len(features_out)+1)
    #sub_data=sub_data.reset_index()
    return sub_data

def apply_style(sub_data):
    return sub_data.style.hide_index().format(
        {"percentage":"{:.2%}","salary_mean":"{:.0f}","salary_median":"{:.0f}","salary_95_min":"{:.0f}","salary_95_max":"{:.0f}"})

In [ ]:
data_pl=get_sub_stats_by_prefix(data,'pl_')

In [ ]:
apply_style(data_pl)

In [ ]:
data_pl.shape

In [ ]:
data_pl=data_pl[['pl_','percentage']].sort_values(by='percentage', ascending=False)
data_pl.reset_index(drop=True, inplace=True)

In [ ]:

data_pl['rank']=list(range(1,24))
data_pl=data_pl[['rank','pl_','percentage']]
#data_pl.style.format({"percentage":"{:.2%}"})
data_pl.style.hide_index().format({"percentage":"{:.2%}"})

In [ ]:
data_pl=data_pl.sort_values(by='percentage')

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = data_pl['pl_']
sizes = data_pl['percentage']
#explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots(figsize=(15, 9))
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title(f"Programming Languages in China, {year}{month:02}")
plt.show()

## Word Cloud

In [ ]:
from wordcloud import WordCloud
wc=WordCloud()
wc_dict = {}
for index, row in data_pl.iterrows():
    wc_dict[row['pl_']]=row['percentage']


wc=wc.generate_from_frequencies(wc_dict)

In [ ]:
wc_dict['c/c++']=0.33
wc=wc.generate_from_frequencies(wc_dict)

In [ ]:
plt.figure(figsize=(10,6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
print(f'{year}年{month}月')